In [ ]:
import time
import numpy as np
import pandas as pd
import os
from os.path import join, abspath, dirname, realpath
import sys
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date, timedelta
from dateutil import relativedelta
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, roc_auc_score
from matplotlib.colors import LinearSegmentedColormap


from dateutil.relativedelta import relativedelta

from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.metrics import auc
import argparse
import csv


import multiprocessing as mp
from multiprocessing import Process

from functools import partial
from skopt import gp_minimize
from skopt import space

from functions import *
from Train_bayes import *
import seaborn as sns
from collections import Counter
from matplotlib.colors import LinearSegmentedColormap


In [ ]:
df = pd.read_csv('df_clean_plus.csv', sep=',', index_col=False, dtype='unicode')#, error_bad_lines=False
df['date']=pd.to_datetime(df['date'])


In [ ]:
#Data statistic metrics
x=df[df.imputation=='False']
data=[]
for i in np.unique(x.patient_id):
    data.append(x[x.patient_id==i].shape[0])
data=np.array(data)
print(np.mean(data),np.std(data),sum(data),min(data))
np.std(np.array([2,100,112,122,123,100]))

In [ ]:
x=df.groupby(['patient_id','order']).size().reset_index(name='Count')
seq_each_patient=[]
for p in np.unique(x.patient_id):
    seq_each_patient.append(len(x[(x.patient_id==p)]))
    

In [ ]:
#count sequences per patient
count_dict=Counter(seq_each_patient)
for k,v in count_dict.items():
    print(k,'occurs', v, 'times')

In [ ]:
#use of imputation as dummy variable to create the plot with missing values
#big gaps had to be different colour so I use different value for the imputation of big gaps
#this is used only here and not for the training or testing of the models

pat_id=np.unique(df['patient_id'])
order=0
a=1
for j,patient in enumerate(pat_id):

    pat_data=df[df['patient_id']==patient].sort_values(by='date')
    pat_data.reset_index(drop=True, inplace=True)
    p=pat_data['date']
    day_before_gap=[]
    days_to_add=[]
    #iteration to the days of each patients to find gaps inbetween
    for i in np.arange(1,p.shape[0]):
        gap = relativedelta(p[i],p[i-1])
        if gap.months>0 or gap.years>0:
            
            for z in np.arange(1,(p[i]-p[i-1]).days):#+ 1):

                l=p[i-1]+timedelta(days=int(z))
                days_to_add.append(l)

    df1 = pd.DataFrame({
                    "patient_id": len(days_to_add)*[patient],
                    "date": days_to_add,
                    "imputation": len(days_to_add)*[2]
                })
                
    df = pd.concat([df,df1])
    


In [ ]:
df['imputation']=df['imputation'].replace('False',0)
df['imputation']=df['imputation'].replace('True',1)
df['date']=pd.to_datetime(df['date'])
df=df.sort_values(['patient_id','date'])
df=df.reset_index(drop=True)


# Patients with big gaps

In [ ]:
#to make graphs I used data after '2021-10-01'
df1=df.loc[(df['date']>pd.to_datetime('2021-10-01'))]

#only patients with imputation==2-> big gaps of data
p=np.unique(df1.loc[df1['imputation']==2]['patient_id'])
df1=df1.loc[df1['patient_id'].isin(p),:]



In [ ]:
#use of imputation as dummy variable 
p=np.unique(df1['date'])
max_date=max(df1['date'])
min_date=min(df1['date'])

for patient in np.unique(df1.patient_id): 
    days=[]
    patient_last_day=max(df.loc[df['patient_id']==patient]['date'])
    patient_first_day=min(df.loc[df['patient_id']==patient]['date'])

    
    for i in range((max_date-patient_last_day).days):
        day= patient_last_day + timedelta(days=i+1)

        if df1[(df1['patient_id']==patient)&(df1['date']==day)].empty:
            days.append(day)
            
    for i in range((patient_first_day-min_date).days):
        day= min_date + timedelta(days=i)

        if df1[(df1['patient_id']==patient)&(df1['date']==day)].empty:
            days.append(day)
    
    df2 = pd.DataFrame({
                    "patient_id": len(days)*[patient],
                    "date": days,
                    "imputation": len(days)*[3]
                })

    
    df1 = pd.concat([df1,df2])


In [ ]:
df1['date']=df1['date'].dt.strftime('%Y-%m-%d')
dfp = df1.pivot_table(index='patient_id', columns='date', values='imputation')
df1

In [ ]:
# drawing the plot
sns.set(rc={'axes.facecolor':'w'})
fig, ax = plt.subplots(figsize=(10, 6))
#with gaps
cmap_rg=LinearSegmentedColormap.from_list('rg',['g','w','r','lightgrey'], N=256)

x=sns.heatmap(dfp, ax=ax,cmap=cmap_rg,cbar=False,linewidth=0.1,linecolor='lightgrey',yticklabels=True)#,cmap="RdYlGn_r",linewidths=0)#cmap='GnBu',ax=ax)


plt.xticks(fontsize=8)

plt.yticks(fontsize=6)
plt.ylabel('Patients_id')
plt.xlabel('Date')
plt.show()

# Patients without big gaps

In [ ]:
df1=df.loc[(df['date']>pd.to_datetime('2021-10-01'))]#& (df['date']<pd.to_datetime('2021-06-15'))]

#only patients with imputation==2-> big gaps of data
p=np.unique(df1.loc[df1['imputation']==2]['patient_id'])
df1=df1.loc[~df1['patient_id'].isin(p),:]
p_nogap=np.unique(df1.patient_id)
random.seed(1995)
random_pick=random.sample(range(0, np.unique(df1.patient_id).shape[0]), 47)

df1=df1.loc[df1['patient_id'].isin(p_nogap[random_pick]),:]
np.unique(df1.patient_id).shape

In [ ]:
#preparation of te dummy variable
#exactly the same as previously 

p=np.unique(df1['date'])
max_date=max(df1['date'])
min_date=min(df1['date'])

for patient in np.unique(df1.patient_id): 
    days=[]
    patient_last_day=max(df.loc[df['patient_id']==patient]['date'])
    patient_first_day=min(df.loc[df['patient_id']==patient]['date'])

    
    for i in range((max_date-patient_last_day).days):
        day= patient_last_day + timedelta(days=i+1)

        if df1[(df1['patient_id']==patient)&(df1['date']==day)].empty:
            days.append(day)
            
    for i in range((patient_first_day-min_date).days):
        day= min_date + timedelta(days=i)

        if df1[(df1['patient_id']==patient)&(df1['date']==day)].empty:
            days.append(day)
    
    df2 = pd.DataFrame({
                    "patient_id": len(days)*[patient],
                    "date": days,
                    "imputation": len(days)*[3]
                })

    
    df1 = pd.concat([df1,df2])


In [ ]:
df1['date']=df1['date'].dt.strftime('%Y-%m-%d')
dfp = df1.pivot_table(index='patient_id', columns='date', values='imputation')
df1

In [ ]:

# drawing the plot
sns.set(rc={'axes.facecolor':'w'})
fig, ax = plt.subplots(figsize=(10, 6))
#with gaps
cmap_rg=LinearSegmentedColormap.from_list('rg',['g','w','r','lightgrey'], N=256)

x=sns.heatmap(dfp, ax=ax,cmap=cmap_rg,cbar=False,linewidth=0.1,linecolor='lightgrey',yticklabels=True)#,cmap="RdYlGn_r",linewidths=0)#cmap='GnBu',ax=ax)


plt.xticks(fontsize=8)

plt.yticks(fontsize=6)
plt.ylabel('Patients_id')
plt.xlabel('Date')
plt.show()